# Training PerAct Agent on handoversim data

In [ ]:
import numpy as np
np.bool = np.bool_ # bad trick to fix numpy version issue :(
import os
import sys
from natsort import natsorted

sys.path = [p for p in sys.path if '/peract/' not in p]

# Set `PYOPENGL_PLATFORM=egl` for pyrender visualizations
os.environ["DISPLAY"] = ":0"
os.environ["PYOPENGL_PLATFORM"] = "egl"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,3" #"0,1,2,3" # Depends on your computer and available GPUs

In [ ]:
# import itertools

# available_cameras = [f"view_{camera_i}" for camera_i in range(12)]

# # Grid search
# grid = {
#     'fill_replay_setting': ["crop"],#, "standard", "uniform"],
#     'cameras': [available_cameras[6:][::2],
#                 available_cameras[6:],
#                 available_cameras[:6],
#                 available_cameras[1::2][:3]+available_cameras[0::2][3:],
#                 available_cameras[:6][-2:]+available_cameras[6:][:4]],
#     'RGB_AUGMENTATION': ['None'],#,'partial','full'],
#     'demo_augm_n': [5],
#     'keypoint_approach': [True],#, False],
#     'data': ['handoversim_v4'],
#     # 'only_learn_approach': [True]
# }
# # Loop over al grid search combinations
# counter = 0
# lst_settings = []
# for values in itertools.product(*grid.values()):
    
#     point = dict(zip(grid.keys(), values))
#     # merge the general settings
#     settings = {**point}
#     lst_settings.append(settings)
#     print(counter, settings)
#     counter += 1

In [ ]:
# from notebook_helpers.constants import BATCH_SIZE
# from notebook_helpers.build_replay import load_replay_buffer
# from notebook_helpers.build_training import build_agent, agent_training

# if BATCH_SIZE != 2:
#     raise ValueError("BATCH_SIZE must be 2 for this notebook")

# # Loop over al grid search combinations: and run
# for run_settings in lst_settings:
#     print(run_settings)

#     # --- load data and arguments: from constants ---

#     # STATIC VALUES USED IN BELOW FUNCTION: SETTING THEM AS GLOBAL FOR FURTHER USE
#     # ___DATA___

#     # Data constants

#     EPISODE_FOLDER = 'episode%d'
#     SETUP = "s1" # Options: "s1"

#     # Data folder
#     WORKSPACE_DIR = os.getcwd()
#     DATA_FOLDER = os.path.join(WORKSPACE_DIR, "task_data", "handoversim_v4")
#     DATA_FOLDER = DATA_FOLDER.replace("/peract_colab", "")
    
#     REFERENCE_TASKS_DIR = os.path.join(DATA_FOLDER, "train_{}".format(SETUP))
#     ALL_TASKS = os.listdir(REFERENCE_TASKS_DIR)
#     for TASK_i, TASK in enumerate(ALL_TASKS):
#         if not TASK in ["handing_over_pitcher_base"]:
#             continue

#         EPISODES_FOLDER = os.path.join(TASK, "all_variations", "episodes")

#         train_data_path = os.path.join(DATA_FOLDER, f"train_{SETUP}", EPISODES_FOLDER)
#         TRAIN_INDEXES = [int(episode_nr.replace("episode", "")) for episode_nr in natsorted(os.listdir(train_data_path))]
#         test_data_path = os.path.join(DATA_FOLDER, f"val_{SETUP}", EPISODES_FOLDER)
#         TEST_INDEXES = [int(episode_nr.replace("episode", "")) for episode_nr in natsorted(os.listdir(test_data_path))]

#         print(f"\nTASK: ({TASK_i} / {len(ALL_TASKS)}): {TASK} ______________________")
#         print(f"TRAIN | Total #: {len(TRAIN_INDEXES)}, indices: {TRAIN_INDEXES}")
#         print(f"TEST | Total #: {len(TEST_INDEXES)}, indices: {TEST_INDEXES}")

#         train_data_iter, test_data_iter = load_replay_buffer(run_settings,
#                                                              WORKSPACE_DIR, SETUP, EPISODE_FOLDER,
#                                                              TASK,
#                                                              train_data_path, test_data_path, TRAIN_INDEXES, TEST_INDEXES)
#         peract_agent = build_agent(run_settings)
#         agent_training(run_settings, peract_agent, train_data_iter, test_data_iter, WORKSPACE_DIR, TASK)

In [ ]:
# # For single (test) run
# available_cameras = [f"view_{camera_i}" for camera_i in range(3)]
# run_settings = {
#     'fill_replay_setting': "crop",
#     'cameras': available_cameras,
#     'RGB_AUGMENTATION': 'partial',
#     'keypoint_approach': True,
#     'demo_augm_n': 5,
#     'only_learn_approach': True
# }
# print(run_settings)

# train_data_iter, test_data_iter = load_replay_buffer(run_settings)
# peract_agent = build_agent(run_settings)
# agent_training(run_settings, peract_agent, train_data_iter, test_data_iter)

In [ ]:
# Testing object specific grasp locations (mug: handle vs. rim)

import itertools

available_cameras = [f"view_{camera_i}" for camera_i in range(12)]

# Grid search
grid = {
    'fill_replay_setting': ["crop"],#, "standard", "uniform"],
    'cameras': [#available_cameras[6:][::2]],
                # available_cameras[6:],
                # available_cameras[:6],
                # available_cameras[1::2][:3]+available_cameras[0::2][3:],
                available_cameras[:6][-2:]+available_cameras[6:][:4]],
    'RGB_AUGMENTATION': ['None'],#,'partial','full'],
    'demo_augm_n': [5],
    'keypoint_approach': [True],#, False],
    'data': ['handoversim_v4'],
    'test': ['mug - grasp the handle'],
    'loss_computation': ['total = trans + rot'],
    'validation': ['remove out of distribution episode 466']
    # 'only_learn_approach': [True]
}
# Loop over al grid search combinations
counter = 0
lst_settings = []
for values in itertools.product(*grid.values()):
    
    point = dict(zip(grid.keys(), values))
    # merge the general settings
    settings = {**point}
    lst_settings.append(settings)
    print(counter, settings)
    counter += 1

In [ ]:
# Testing object specific grasp locations (mug: handle vs. rim)

from notebook_helpers.constants import BATCH_SIZE
from notebook_helpers.build_replay import load_replay_buffer
from notebook_helpers.build_training import build_agent, agent_training

if BATCH_SIZE != 2:
    raise ValueError("BATCH_SIZE must be 2 for this notebook")

# Loop over al grid search combinations: and run
for run_settings in lst_settings:
    print(run_settings)

    # --- load data and arguments: from constants ---

    # STATIC VALUES USED IN BELOW FUNCTION: SETTING THEM AS GLOBAL FOR FURTHER USE
    # ___DATA___

    # Data constants

    EPISODE_FOLDER = 'episode%d'
    SETUP = "s1" # Options: "s1"

    # Data folder
    WORKSPACE_DIR = os.getcwd()
    DATA_FOLDER = os.path.join(WORKSPACE_DIR, "task_data", "handoversim_v4")
    DATA_FOLDER = DATA_FOLDER.replace("/peract_colab", "")
    
    REFERENCE_TASKS_DIR = os.path.join(DATA_FOLDER, "train_{}".format(SETUP))
    ALL_TASKS = os.listdir(REFERENCE_TASKS_DIR)
    for TASK_i, TASK in enumerate(ALL_TASKS):
        if not TASK in ["handing_over_mug"]:
            continue

        EPISODES_FOLDER = os.path.join(TASK, "all_variations", "episodes")

        train_data_path = os.path.join(DATA_FOLDER, f"train_{SETUP}", EPISODES_FOLDER)
        test_data_path = os.path.join(DATA_FOLDER, f"train_{SETUP}", EPISODES_FOLDER)
        TRAIN_INDEXES = [66, 266, 268, 269, 368] # handle
        TEST_INDEXES = [465]#, 466] # handle

        # TRAIN_INDEXES = TRAIN_INDEXES + [167, 169, 265, 365, 366, 367, 369, 468, 566, 567, 568, 569] # rim
        # TEST_INDEXES = TEST_INDEXES + [966, 967, 968] # rim

        print(f"\nTASK: ({TASK_i} / {len(ALL_TASKS)}): {TASK} ______________________")
        print(f"TRAIN | Total #: {len(TRAIN_INDEXES)}, indices: {TRAIN_INDEXES}")
        print(f"TEST | Total #: {len(TEST_INDEXES)}, indices: {TEST_INDEXES}")

        train_data_iter, test_data_iter = load_replay_buffer(run_settings,
                                                             WORKSPACE_DIR, SETUP, EPISODE_FOLDER,
                                                             TASK,
                                                             train_data_path, test_data_path, TRAIN_INDEXES, TEST_INDEXES)
        peract_agent = build_agent(run_settings)
        agent_training(run_settings, peract_agent, train_data_iter, test_data_iter, WORKSPACE_DIR, TASK)